# <span style='font-family:"Times New Roman"'> <span styel=''>**PAIR ANALYSIS**
*Emile Cohen*
    
 *July 2020*

**Goal:** In this notebook, we want to have an overview of paired samples and the state evolution.

---

In [269]:
%run -i '../../utils/setup_environment.ipy'

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import fisher_exact, ranksums, chi2, norm
from statsmodels.sandbox.stats.multicomp import multipletests
import matplotlib.gridspec as gridspec

data_path = '../../data/'

Setup environment... done!


<span style="color:green">✅ Working on **mskimpact_env** conda environment.</span>

# Loading the Masters

In [270]:
master_no_wgd = non_wgd_load_and_cut(data_path + 'impact-facets-tp53/processed/no_wgd/master_no_wgd.pkl')
master_wgd = pd.read_pickle(data_path + 'impact-facets-tp53/processed/wgd/master_wgd.pkl')

maf_cohort_nowgd = pd.read_csv(data_path + 'impact-facets-tp53/processed/no_wgd/maf_cohort_nowgd.txt', sep='\t').drop('Unnamed: 0', axis=1)
maf_cohort_wgd = pd.read_csv(data_path + 'impact-facets-tp53/processed/wgd/maf_cohort_wgd.txt', sep='\t').drop('Unnamed: 0', axis=1)

In [271]:
master_no_wgd['wgd'] = 0
master_wgd['wgd'] = 1

In [8]:
master_no_wgd.head(5)

,Sample_Id,Tumor_Id,Patient_Id,Cancer_Type,Cancer_Type_Detailed,Patient_Current_Age,Sex,Sample_Type,purity,ploidy,Overall_Survival_Months,Overall_Survival_Status,MSI_Score,MSI_Type,TMB_Score,tp53_key_1,tp53_vc_1,tp53_ccf_1,tp53_vaf_1,tp53_HGVSp_1,tp53_spot_1,tp53_vt_1,tp53_key_2,tp53_vc_2,tp53_ccf_2,tp53_vaf_2,tp53_HGVSp_2,tp53_spot_2,tp53_vt_2,tp53_key_3,tp53_vc_3,tp53_ccf_3,tp53_vaf_3,tp53_HGVSp_3,tp53_spot_3,tp53_vt_3,tp53_key_4,tp53_vc_4,tp53_ccf_4,tp53_vaf_4,tp53_HGVSp_4,tp53_spot_4,tp53_vt_4,tp53_key_5,tp53_vc_5,tp53_ccf_5,tp53_vaf_5,tp53_HGVSp_5,tp53_spot_5,tp53_vt_5,tp53_count,tp53_tcn,tp53_mcn,tp53_lcn,tp53_seg_length,tp53_cn_state,tp53_cf,wgd,gene_count,mutation_count,driver_gene_count,driver_mutation_count,snv_driver_mutation_count,indel_driver_mutation_count,max_vaf,tp53_exp_nb_1,tp53_exp_nb_2,tp53_exp_nb_3,tp53_exp_nb_4,tp53_exp_nb_5,tp53_res_1,tp53_res_2,tp53_res_3,tp53_res_4,tp53_res_5,tp53_vc_group_1,tp53_vc_group_2,tp53_vc_group_3,tp53_vc_group_4,tp53_vc_group_5,tp53_vt_group_1,tp53_vt_group_2,tp53_vt_group_3,tp53_vt_group_4,tp53_vt_group_5,tp53_first_group,tp53_group,tp53_res_group,tp53_loh_status,chr_affected,chr_loss,chr_gain,chr_cnloh,frac_genome_altered
0,P-0034223-T01-IM6_P-0034223-N01-IM6,P-0034223-T01-IM6,P-0034223,Breast Cancer,Invasive Breast Carcinoma,63.0,Female,Metastasis,0.946448,2.241830,NaN,LIVING,0.55,Stable,5.3,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,80668592,DIPLOID,1.000000,0,6,6,5,5,3,2,0.901899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,3.0,2.0,2.0,0.0,0.080
1,P-0009819-T01-IM5_P-0009819-N01-IM5,P-0009819-T01-IM5,P-0009819,Prostate Cancer,Prostate Adenocarcinoma,72.0,Male,Primary,0.278140,2.681075,23.441,LIVING,0.00,Stable,1.0,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,80668300,DIPLOID,1.000000,0,1,1,0,0,0,0,0.148014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,4.0,4.0,0.0,0.0,0.137
2,P-0027408-T01-IM6_P-0027408-N01-IM6,P-0027408-T01-IM6,P-0027408,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,67.0,Female,Metastasis,0.275073,1.811066,22.586,LIVING,0.27,Stable,17.6,P-0027408-T01-IM6-17_7578409_CT_TC,Missense_Mutation,1.0,0.168901,p.Arg174Glu,174,DNP,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,1,1,1.0,0.0,26256025,HETLOSS,0.275073,0,17,19,4,4,3,1,0.192475,1.059141,NaN,NaN,NaN,NaN,-0.059141,NaN,NaN,NaN,NaN,missense,None,None,None,None,SNV,None,None,None,None,LOSS,>=1_LOSS,no_tp53_res,True,12.0,11.0,3.0,0.0,0.444
3,P-0006554-T01-IM5_P-0006554-N01-IM5,P-0006554-T01-IM5,P-0006554,Glioma,Anaplastic Oligodendroglioma,55.0,Female,Primary,0.775152,1.910719,26.170,LIVING,1.30,Stable,46.2,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,40254480,DIPLOID,1.000000,0,39,47,6,6,5,1,0.706897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,5.0,4.0,1.0,0.0,0.173
4,P-0036980-T01-IM6_P-0036980-N01-IM6,P-0036980-T01-IM6,P-0036980,Hepatobiliary Cancer,Intrahepatic Cholangiocarcinoma,54.0,Female,Primary,0.845207,2.267573,9.501,LIVING,0.45,Stable,4.4,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,80668381,DIPLOID,1.000000,0,5,5,3,3,3,0,0.814815,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,4.0,4.0,1.0,0.0,0.187


# Analysis

## Aggregation
We concatenate the two masters to have a big master with all patients and to be able to see the state evolution

In [272]:
master_no_wgd_sub = master_no_wgd[['Sample_Id', 'Patient_Id', 'Cancer_Type']]

In [273]:
master_total = pd.concat([master_no_wgd, master_wgd])
master_total.head()

,Sample_Id,Tumor_Id,Patient_Id,Cancer_Type,Cancer_Type_Detailed,Patient_Current_Age,Sex,Sample_Type,purity,ploidy,Overall_Survival_Months,Overall_Survival_Status,MSI_Score,MSI_Type,TMB_Score,tp53_key_1,tp53_vc_1,tp53_ccf_1,tp53_vaf_1,tp53_HGVSp_1,tp53_spot_1,tp53_vt_1,tp53_key_2,tp53_vc_2,tp53_ccf_2,tp53_vaf_2,tp53_HGVSp_2,tp53_spot_2,tp53_vt_2,tp53_key_3,tp53_vc_3,tp53_ccf_3,tp53_vaf_3,tp53_HGVSp_3,tp53_spot_3,tp53_vt_3,tp53_key_4,tp53_vc_4,tp53_ccf_4,tp53_vaf_4,tp53_HGVSp_4,tp53_spot_4,tp53_vt_4,tp53_key_5,tp53_vc_5,tp53_ccf_5,tp53_vaf_5,tp53_HGVSp_5,tp53_spot_5,tp53_vt_5,tp53_count,tp53_tcn,tp53_mcn,tp53_lcn,tp53_seg_length,tp53_cn_state,tp53_cf,wgd,gene_count,mutation_count,driver_gene_count,driver_mutation_count,snv_driver_mutation_count,indel_driver_mutation_count,max_vaf,tp53_exp_nb_1,tp53_exp_nb_2,tp53_exp_nb_3,tp53_exp_nb_4,tp53_exp_nb_5,tp53_res_1,tp53_res_2,tp53_res_3,tp53_res_4,tp53_res_5,tp53_vc_group_1,tp53_vc_group_2,tp53_vc_group_3,tp53_vc_group_4,tp53_vc_group_5,tp53_vt_group_1,tp53_vt_group_2,tp53_vt_group_3,tp53_vt_group_4,tp53_vt_group_5,tp53_first_group,tp53_group,tp53_res_group,tp53_loh_status,chr_affected,chr_loss,chr_gain,chr_cnloh,frac_genome_altered,tp53_bi_state,tp53_state_wgd,tp53_mut_loh_group,prewgd_tp53_group_1,prewgd_tp53_group_2
0,P-0034223-T01-IM6_P-0034223-N01-IM6,P-0034223-T01-IM6,P-0034223,Breast Cancer,Invasive Breast Carcinoma,63.0,Female,Metastasis,0.946448,2.241830,NaN,LIVING,0.55,Stable,5.3,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,80668592,DIPLOID,1.000000,0,6,6,5,5,3,2,0.901899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,3.0,2.0,2.0,0.0,0.080,NaN,NaN,NaN,NaN,NaN
1,P-0009819-T01-IM5_P-0009819-N01-IM5,P-0009819-T01-IM5,P-0009819,Prostate Cancer,Prostate Adenocarcinoma,72.0,Male,Primary,0.278140,2.681075,23.441,LIVING,0.00,Stable,1.0,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,80668300,DIPLOID,1.000000,0,1,1,0,0,0,0,0.148014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,4.0,4.0,0.0,0.0,0.137,NaN,NaN,NaN,NaN,NaN
2,P-0027408-T01-IM6_P-0027408-N01-IM6,P-0027408-T01-IM6,P-0027408,Non-Small Cell Lung Cancer,Non-Small Cell Lung Cancer,67.0,Female,Metastasis,0.275073,1.811066,22.586,LIVING,0.27,Stable,17.6,P-0027408-T01-IM6-17_7578409_CT_TC,Missense_Mutation,1.0,0.168901,p.Arg174Glu,174,DNP,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,1,1,1.0,0.0,26256025,HETLOSS,0.275073,0,17,19,4,4,3,1,0.192475,1.059141,NaN,NaN,NaN,NaN,-0.059141,NaN,NaN,NaN,NaN,missense,None,None,None,None,SNV,None,None,None,None,LOSS,>=1_LOSS,no_tp53_res,True,12.0,11.0,3.0,0.0,0.444,NaN,NaN,NaN,NaN,NaN
3,P-0006554-T01-IM5_P-0006554-N01-IM5,P-0006554-T01-IM5,P-0006554,Glioma,Anaplastic Oligodendroglioma,55.0,Female,Primary,0.775152,1.910719,26.170,LIVING,1.30,Stable,46.2,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,40254480,DIPLOID,1.000000,0,39,47,6,6,5,1,0.706897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,5.0,4.0,1.0,0.0,0.173,NaN,NaN,NaN,NaN,NaN
4,P-0036980-T01-IM6_P-0036980-N01-IM6,P-0036980-T01-IM6,P-0036980,Hepatobiliary Cancer,Intrahepatic Cholangiocarcinoma,54.0,Female,Primary,0.845207,2.267573,9.501,LIVING,0.45,Stable,4.4,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,80668381,DIPLOID,1.000000,0,5,5,

## General Numbers

In [274]:
h = pd.DataFrame(master_total[['Patient_Id', 'Cancer_Type', 'wgd']].groupby(['Patient_Id']).size())
get_groupby(h,0, '# of samples per patient')

,# of samples per patient
0,
1,19262
2,1119
3,112
4,9
5,2


* We have 1242 paired samples

In [275]:
paired_patients = h[h[0]>=2].index.tolist()
master_paired = master_total[master_total['Patient_Id'].isin(paired_patients)]
master_paired['T'] = master_paired.Tumor_Id.str[12]

master_paired.head(10)

,Sample_Id,Tumor_Id,Patient_Id,Cancer_Type,Cancer_Type_Detailed,Patient_Current_Age,Sex,Sample_Type,purity,ploidy,Overall_Survival_Months,Overall_Survival_Status,MSI_Score,MSI_Type,TMB_Score,tp53_key_1,tp53_vc_1,tp53_ccf_1,tp53_vaf_1,tp53_HGVSp_1,tp53_spot_1,tp53_vt_1,tp53_key_2,tp53_vc_2,tp53_ccf_2,tp53_vaf_2,tp53_HGVSp_2,tp53_spot_2,tp53_vt_2,tp53_key_3,tp53_vc_3,tp53_ccf_3,tp53_vaf_3,tp53_HGVSp_3,tp53_spot_3,tp53_vt_3,tp53_key_4,tp53_vc_4,tp53_ccf_4,tp53_vaf_4,tp53_HGVSp_4,tp53_spot_4,tp53_vt_4,tp53_key_5,tp53_vc_5,tp53_ccf_5,tp53_vaf_5,tp53_HGVSp_5,tp53_spot_5,tp53_vt_5,tp53_count,tp53_tcn,tp53_mcn,tp53_lcn,tp53_seg_length,tp53_cn_state,tp53_cf,wgd,gene_count,mutation_count,driver_gene_count,driver_mutation_count,snv_driver_mutation_count,indel_driver_mutation_count,max_vaf,tp53_exp_nb_1,tp53_exp_nb_2,tp53_exp_nb_3,tp53_exp_nb_4,tp53_exp_nb_5,tp53_res_1,tp53_res_2,tp53_res_3,tp53_res_4,tp53_res_5,tp53_vc_group_1,tp53_vc_group_2,tp53_vc_group_3,tp53_vc_group_4,tp53_vc_group_5,tp53_vt_group_1,tp53_vt_group_2,tp53_vt_group_3,tp53_vt_group_4,tp53_vt_group_5,tp53_first_group,tp53_group,tp53_res_group,tp53_loh_status,chr_affected,chr_loss,chr_gain,chr_cnloh,frac_genome_altered,tp53_bi_state,tp53_state_wgd,tp53_mut_loh_group,prewgd_tp53_group_1,prewgd_tp53_group_2,T
5,P-0001315-T01-IM3_P-0001315-N01-IM3,P-0001315-T01-IM3,P-0001315,Gastrointestinal Stromal Tumor,Gastrointestinal Stromal Tumor,42.0,Male,Metastasis,0.934522,2.093267,65.030,LIVING,2.33,Stable,6.7,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,30178359,DIPLOID,1.000000,0,5,6,3,4,1,3,0.902778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,10.0,5.0,3.0,2.0,0.356,NaN,NaN,NaN,NaN,NaN,1
6,P-0001315-T02-IM5_P-0001315-N01-IM5,P-0001315-T02-IM5,P-0001315,Gastrointestinal Stromal Tumor,Gastrointestinal Stromal Tumor,42.0,Male,Metastasis,0.786007,2.072039,65.030,LIVING,2.46,Stable,4.9,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,30185687,DIPLOID,1.000000,0,5,6,3,4,1,3,0.687719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,10.0,5.0,3.0,2.0,0.355,NaN,NaN,NaN,NaN,NaN,2
8,P-0036570-T01-IM6_P-0036570-N01-IM6,P-0036570-T01-IM6,P-0036570,Ovarian Cancer,High-Grade Serous Ovarian Cancer,71.0,Female,Primary,0.783647,2.499971,13.611,LIVING,1.05,Stable,1.8,P-0036570-T01-IM6-17_7578236_A_G,Missense_Mutation,1.000,0.718605,p.Tyr205His,205,SNP,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,1,1,1.0,0.0,37750850,HETLOSS,0.783647,0,1,1,1,1,0,1,0.718605,1.115396,NaN,NaN,NaN,NaN,-0.115396,NaN,NaN,NaN,NaN,missense,None,None,None,None,SNV,None,None,None,None,LOSS,>=1_LOSS,no_tp53_res,True,6.0,2.0,3.0,1.0,0.296,NaN,NaN,NaN,NaN,NaN,1
9,P-0036570-T02-IM6_P-0036570-N01-IM6,P-0036570-T02-IM6,P-0036570,Non-Small Cell Lung Cancer,Lung Adenocarcinoma,71.0,Female,Primary,0.250258,1.714611,13.611,LIVING,0.00,Stable,3.5,P-0036570-T02-IM6-17_7577529_A_T,Missense_Mutation,1.000,0.156652,p.Ile251Asn,251,SNP,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,1,1,1.0,0.0,14319106,HETLOSS,0.250258,0,3,3,0,0,0,0,0.161290,1.095276,NaN,NaN,NaN,NaN,-0.095276,NaN,NaN,NaN,NaN,missense,None,None,None,None,SNV,None,None,None,None,LOSS,>=1_LOSS,no_tp53_res,True,14.0,8.0,6.0,1.0,0.620,NaN,NaN,NaN,NaN,NaN,2
10,P-0023458-T01-IM6_P-0023458-N01-IM6,P-0023458-T01-IM6,P-0023458,Endometrial Cancer,Uterine Mixed Endometrial Carcinoma,72.0,Female,Primary,0.269254,2.525672,25.775,LIVING,0.40,Stable,1.8,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,Na

In [276]:
master_paired[master_paired['Patient_Id'] == 'P-0001315']

,Sample_Id,Tumor_Id,Patient_Id,Cancer_Type,Cancer_Type_Detailed,Patient_Current_Age,Sex,Sample_Type,purity,ploidy,Overall_Survival_Months,Overall_Survival_Status,MSI_Score,MSI_Type,TMB_Score,tp53_key_1,tp53_vc_1,tp53_ccf_1,tp53_vaf_1,tp53_HGVSp_1,tp53_spot_1,tp53_vt_1,tp53_key_2,tp53_vc_2,tp53_ccf_2,tp53_vaf_2,tp53_HGVSp_2,tp53_spot_2,tp53_vt_2,tp53_key_3,tp53_vc_3,tp53_ccf_3,tp53_vaf_3,tp53_HGVSp_3,tp53_spot_3,tp53_vt_3,tp53_key_4,tp53_vc_4,tp53_ccf_4,tp53_vaf_4,tp53_HGVSp_4,tp53_spot_4,tp53_vt_4,tp53_key_5,tp53_vc_5,tp53_ccf_5,tp53_vaf_5,tp53_HGVSp_5,tp53_spot_5,tp53_vt_5,tp53_count,tp53_tcn,tp53_mcn,tp53_lcn,tp53_seg_length,tp53_cn_state,tp53_cf,wgd,gene_count,mutation_count,driver_gene_count,driver_mutation_count,snv_driver_mutation_count,indel_driver_mutation_count,max_vaf,tp53_exp_nb_1,tp53_exp_nb_2,tp53_exp_nb_3,tp53_exp_nb_4,tp53_exp_nb_5,tp53_res_1,tp53_res_2,tp53_res_3,tp53_res_4,tp53_res_5,tp53_vc_group_1,tp53_vc_group_2,tp53_vc_group_3,tp53_vc_group_4,tp53_vc_group_5,tp53_vt_group_1,tp53_vt_group_2,tp53_vt_group_3,tp53_vt_group_4,tp53_vt_group_5,tp53_first_group,tp53_group,tp53_res_group,tp53_loh_status,chr_affected,chr_loss,chr_gain,chr_cnloh,frac_genome_altered,tp53_bi_state,tp53_state_wgd,tp53_mut_loh_group,prewgd_tp53_group_1,prewgd_tp53_group_2,T
5,P-0001315-T01-IM3_P-0001315-N01-IM3,P-0001315-T01-IM3,P-0001315,Gastrointestinal Stromal Tumor,Gastrointestinal Stromal Tumor,42.0,Male,Metastasis,0.934522,2.093267,65.03,LIVING,2.33,Stable,6.7,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,30178359,DIPLOID,1.0,0,5,6,3,4,1,3,0.902778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,10.0,5.0,3.0,2.0,0.356,NaN,NaN,NaN,NaN,NaN,1
6,P-0001315-T02-IM5_P-0001315-N01-IM5,P-0001315-T02-IM5,P-0001315,Gastrointestinal Stromal Tumor,Gastrointestinal Stromal Tumor,42.0,Male,Metastasis,0.786007,2.072039,65.03,LIVING,2.46,Stable,4.9,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,30185687,DIPLOID,1.0,0,5,6,3,4,1,3,0.687719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,10.0,5.0,3.0,2.0,0.355,NaN,NaN,NaN,NaN,NaN,2


## Numbers per cancer type

In [277]:
get_groupby(master_paired, 'Cancer_Type', '# of paired samples').sort_values(by='# of paired samples', ascending=False).head(15)

,# of paired samples
Cancer_Type,
Breast Cancer,604
Non-Small Cell Lung Cancer,492
Glioma,201
Prostate Cancer,180
Colorectal Cancer,154
Bladder Cancer,104
Gastrointestinal Stromal Tumor,98
Melanoma,67
Renal Cell Carcinoma,67


# State Evolution

In [278]:
h = pd.DataFrame(master_total[['Patient_Id', 'Cancer_Type', 'wgd']].groupby(['Patient_Id']).size())
state_evolution = pd.DataFrame(h[h[0]==2])
state_evolution.columns=['num_samples']
state_evolution['Patient_Id'] = state_evolution.index

In [279]:
test_evolution= state_evolution[0:5]
test_evolution

,num_samples,Patient_Id
Patient_Id,,
P-0000037,2,P-0000037
P-0000138,2,P-0000138
P-0000152,2,P-0000152
P-0000167,2,P-0000167
P-0000196,2,P-0000196


In [294]:
def get_state_evolution(x, master: pd.DataFrame):
    lookup = master[master['Patient_Id'] == x.Patient_Id]
    tumors = lookup['T'].tolist()
    tumors.sort()
    tumors_first = tumors[:2]
    
    sample_1 = lookup[lookup['T'] == tumors_first[0]]
    sample_2 = lookup[lookup['T'] == tumors_first[1]]
    
    if sample_1['wgd'].values[0] == 0 and sample_2['wgd'].values[0] == 0:
        if sample_1['tp53_res_group'].values[0] == 'tp53_res' and sample_2['tp53_res_group'].values[0] == 'tp53_res':
            return 'stable_NOWGDres_NOWGDres'

        elif sample_1['tp53_res_group'].values[0] == 'tp53_res' and sample_2['tp53_res_group'].values[0] == 'no_tp53_res':
            return 'change_NOWGDres_NOWGDnores'

        elif sample_1['tp53_res_group'].values[0] == 'no_tp53_res' and sample_2['tp53_res_group'].values[0] == 'no_tp53_res':
            return 'stable_NOWGDnores_NOWGDnores'

        elif sample_1['tp53_res_group'].values[0] == 'no_tp53_res' and sample_2['tp53_res_group'].values[0] == 'tp53_res':
            return 'change_NOWGDnores_NOWGDres'
        
        
        elif sample_1['tp53_res_group'].values[0] == 'uncertain' and sample_2['tp53_res_group'].values[0] == 'tp53_res':
            return 'change_NOWGDuncertain_NOWGDres'
        
        elif sample_2['tp53_res_group'].values[0] == 'uncertain' and sample_1['tp53_res_group'].values[0] == 'tp53_res':
            return 'change_NOWGDres_NOWGDuncertain'
        
        elif sample_1['tp53_res_group'].values[0] == 'uncertain' and sample_2['tp53_res_group'].values[0] == 'no_tp53_res':
            return 'change_NOWGDuncertain_NOWGDnores'
        
        elif sample_2['tp53_res_group'].values[0] == 'uncertain' and sample_1['tp53_res_group'].values[0] == 'no_tp53_res':
            return 'change_NOWGDnores_NOWGDuncertain'
        
        elif sample_2['tp53_res_group'].values[0] == 'uncertain' and sample_1['tp53_res_group'].values[0] == 'uncertain':
            return 'change_NOWGDuncertain_NOWGDuncertain'
        
        elif sample_1['tp53_res_group'].values[0] == None:
            if sample_2['tp53_res_group'].values[0] == 'no_tp53_res':
                return 'change_NOWGDwt_NOWGDnores'
            elif sample_2['tp53_res_group'].values[0] == 'tp53_res':
                return 'change_NOWGDwt_NOWGDres'
            elif sample_2['tp53_res_group'].values[0] == None:
                return 'stable_NOWGDwt_NOWGDwt'
            
        elif sample_2['tp53_res_group'].values[0] == None:
            if sample_1['tp53_res_group'].values[0] == 'no_tp53_res':
                return 'change_NOWGDnores_NOWGDwt'
            elif sample_1['tp53_res_group'].values[0] == 'tp53_res':
                return 'change_NOWGDres_NOWGDwt'
            elif sample_1['tp53_res_group'].values[0] == None:
                return 'stable_NOWGDwt_NOWGDwt'
                
    
    
    elif sample_1['wgd'].values[0] == 1 and sample_2['wgd'].values[0] == 1:
        if sample_1['prewgd_tp53_group_1'].values[0] == 'tp53_res' and sample_2['prewgd_tp53_group_1'].values[0] == 'tp53_res':
            return 'stable_WGDres_WGDres'
        elif sample_1['prewgd_tp53_group_1'].values[0] == 'tp53_res' and sample_2['prewgd_tp53_group_1'].values[0] == 'bi':
            return 'change_WGDres_WGDnores'
        elif sample_1['prewgd_tp53_group_1'].values[0] == 'bi' and sample_2['prewgd_tp53_group_1'].values[0] == 'tp53_res':
            return 'change_WGDnores_WGDres'
        elif sample_1['prewgd_tp53_group_1'].values[0] == 'bi' and sample_2['prewgd_tp53_group_1'].values[0] == 'bi':
            return 'stable_WGDnores_WGDnores'
        
        else: return 'uncertain WGD'

    elif sample_1['wgd'].values[0] == 1 and sample_2['wgd'].values[0] == 0:
        if sample_2['tp53_res_group'].values[0] == 'tp53_res' and sample_1['prewgd_tp53_group_1'].values[0] == 'tp53_res':
            return 'change_WGDres_NOWGDres'
        elif sample_2['tp53_res_group'].values[0] == 'tp53_res' and sample_1['prewgd_tp53_group_1'].values[0] == 'bi':
            return 'change_WGDres_NOWGDnores'
        elif sample_2['tp53_res_group'].values[0] == 'no_tp53_res' and sample_1['prewgd_tp53_group_1'].values[0] == 'tp53_res':
            return 'change_WGDnores_NOWGDres'
        elif sample_2['tp53_res_group'].values[0] == 'no_tp53_res' and sample_1['prewgd_tp53_group_1'].values[0] == 'bi':
            return 'change_WGDnores_NOWGDnores'

        
        else: return 'wt or uncertain WGD-NOWGD'
        
        return 'change_wgd_nowgd'

    elif sample_1['wgd'].values[0] == 0 and sample_2['wgd'].values[0] == 1:
        if sample_1['tp53_res_group'].values[0] == 'tp53_res' and sample_2['prewgd_tp53_group_1'].values[0] == 'tp53_res':
            return 'change_NOWGDres_WGDres'
        elif sample_1['tp53_res_group'].values[0] == 'tp53_res' and sample_2['prewgd_tp53_group_1'].values[0] == 'bi':
            return 'change_NOWGDres_WGDnores'
        elif sample_1['tp53_res_group'].values[0] == 'no_tp53_res' and sample_2['prewgd_tp53_group_1'].values[0] == 'tp53_res':
            return 'change_NOWGDnores_WGDres'
        elif sample_1['tp53_res_group'].values[0] == 'no_tp53_res' and sample_2['prewgd_tp53_group_1'].values[0] == 'bi':
            return 'change_NOWGDnores_WGDnores'
        
        elif sample_1['tp53_res_group'].values[0] == None:
            if sample_2['prewgd_tp53_group_1'].values[0] == 'tp53_res':
                return 'change_NOWGDwt_WGDres'
            elif sample_2['prewgd_tp53_group_1'].values[0] == 'bi':
                return 'change_NOWGDwt_WGDnores'
            elif sample_2['prewgd_tp53_group_1'].values[0] == None:
                return 'change_NOWGDwt_WGDwt'
        
        else: return 'uncertain NOWGD-WGD'
    
    elif sample_1['tp53_count'].values[0] == 0 and sample_2['tp53_count'].values[0] == 0:
        return 'WT_WT'

def get_state_evolution_simp(x, master: pd.DataFrame):
    state = x.evolution
    if state in ['stable_WGDres_WGDres','stable_NOWGDres_NOWGDres', 'change_NOWGDres_WGDres', 'change_WGDres_NOWGDres']:
        return 'res-res'
    elif state in ['change_NOWGDres_NOWGDnores', 'change_WGDres_WGDnores', 'change_WGDres_NOWGDnores', 'change_NOWGDres_WGDnores']:
        return 'res-nores'
    elif state in ['change_WGDnores_WGDres', 'change_NOWGDnores_WGDres', 'change_WGDnores_NOWGDres']:
        return 'nores-res'
    elif state in ['stable_WGDnores_WGDnores','stable_NOWGDnores_NOWGDnores', 'change_NOWGDnores_WGDnores', 'change_WGDnores_NOWGDnores']:
        return 'nores-nores'
    elif state in ['change_NOWGDwt_NOWGDres', 'change_NOWGDwt_WGDres']:
        return 'wt-res'
    elif state in ['change_NOWGDwt_NOWGDnores', 'change_NOWGDwt_WGDnores']:
        return 'wt-nores'
    elif state in ['change_NOWGDnores_NOWGDres']:
        return 'nores-wt'
    elif state in ['change_NOWGDres_NOWGDres']:
        return 'res-wt'
    elif state in ['stable_NOWGDwt_NOWGDwt', 'change_NOWGDwt_WGDwt']:
        return 'wt-wt'
    else: return 'other'

def get_PM_evolution(x, master: pd.DataFrame):
    lookup = master[master['Patient_Id'] == x.Patient_Id]
    tumors = lookup['T'].tolist()
    tumors.sort()
    tumors_first = tumors[:2]
    
    sample_1 = lookup[lookup['T'] == tumors_first[0]]
    sample_2 = lookup[lookup['T'] == tumors_first[1]]
    
    if sample_1['Sample_Type'].values[0] == 'Primary' and sample_2['Sample_Type'].values[0] == 'Metastasis':
        return 'PM'
    elif sample_1['Sample_Type'].values[0] == 'Primary' and sample_2['Sample_Type'].values[0] == 'Primary':
        return 'PP'
    elif sample_1['Sample_Type'].values[0] == 'Metastasis' and sample_2['Sample_Type'].values[0] == 'Metastasis':
        return 'MM'
    elif sample_1['Sample_Type'].values[0] == 'Metastasis' and sample_2['Sample_Type'].values[0] == 'Primary':
        return 'MP'

def get_mut_evolution(x, master: pd.DataFrame):
    lookup = master[master['Patient_Id'] == x.Patient_Id]
    tumors = lookup['T'].tolist()
    tumors.sort()
    tumors_first = tumors[:2]
    
    sample_1 = lookup[lookup['T'] == tumors_first[0]]
    sample_2 = lookup[lookup['T'] == tumors_first[1]]
    
    return(str(sample_1['tp53_count'].values[0])+'_'+ str(sample_2['tp53_count'].values[0]))


In [295]:
state_evolution['evolution'] = state_evolution.apply(get_state_evolution, master=master_paired, axis=1)
state_evolution['PM_evolution'] = state_evolution.apply(get_PM_evolution, master=master_paired, axis=1)
state_evolution['mut_evolution'] = state_evolution.apply(get_mut_evolution, master=master_paired, axis=1)
state_evolution['evolution_simp'] = state_evolution.apply(get_state_evolution_simp, master=master_paired, axis=1)

In [296]:
get_groupby(state_evolution, 'PM_evolution', 'count').sort_values(by='count', ascending=False)

,count
PM_evolution,
MM,341
PP,339
PM,314
MP,125


In [297]:
get_groupby(state_evolution[state_evolution['PM_evolution'] == 'PM'], 'evolution_simp', 'count').sort_values(by='count', ascending=False)

,count
evolution_simp,
wt-wt,91
nores-nores,80
res-res,64
other,26
wt-res,21
nores-res,12
res-nores,10
wt-nores,9
nores-wt,1


In [298]:
get_groupby(state_evolution[state_evolution['PM_evolution'] == 'PM'], 'evolution', 'count').sort_values(by='count', ascending=False)

,count
evolution,
stable_NOWGDwt_NOWGDwt,91
stable_WGDnores_WGDnores,35
stable_WGDres_WGDres,29
stable_NOWGDnores_NOWGDnores,26
stable_NOWGDres_NOWGDres,24
change_NOWGDnores_WGDnores,16
change_NOWGDwt_WGDres,15
wt or uncertain WGD-NOWGD,12
change_NOWGDres_WGDres,7


In [203]:
# Cancer Types
patients_PM = state_evolution[state_evolution['PM_evolution'] == 'PM'].Patient_Id.tolist()
get_groupby(master_paired[master_paired['Patient_Id'].isin(patients_PM)], 'Cancer_Type', 'count').sort_values(by='count', ascending=False)

,count
Cancer_Type,
Non-Small Cell Lung Cancer,109
Breast Cancer,75
Glioma,70
Prostate Cancer,58
Colorectal Cancer,53
Pancreatic Cancer,27
Gastrointestinal Stromal Tumor,24
Bladder Cancer,20
Endometrial Cancer,19


In [309]:
# Breakdown per cancer type
cancer_type = 'Prostate Cancer'
patients = master_paired[master_paired['Cancer_Type'] == cancer_type].Patient_Id.tolist()
state_evolution_cancer = state_evolution[state_evolution['Patient_Id'].isin(patients)]
get_groupby(state_evolution_cancer[state_evolution_cancer['PM_evolution'] == 'PM'], 'evolution_simp', cancer_type).sort_values(by=cancer_type, ascending=False)

,Prostate Cancer
evolution_simp,
wt-wt,11
res-res,9
nores-nores,7
other,3
nores-res,2


In [222]:
h = master_paired[master_paired['Patient_Id'].isin(patients_PM)]
g=h[h['Cancer_Type'] == 'Glioma']
g[g['Sample_Type'] == 'Metastasis']

,Sample_Id,Tumor_Id,Patient_Id,Cancer_Type,Cancer_Type_Detailed,Patient_Current_Age,Sex,Sample_Type,purity,ploidy,Overall_Survival_Months,Overall_Survival_Status,MSI_Score,MSI_Type,TMB_Score,tp53_key_1,tp53_vc_1,tp53_ccf_1,tp53_vaf_1,tp53_HGVSp_1,tp53_spot_1,tp53_vt_1,tp53_key_2,tp53_vc_2,tp53_ccf_2,tp53_vaf_2,tp53_HGVSp_2,tp53_spot_2,tp53_vt_2,tp53_key_3,tp53_vc_3,tp53_ccf_3,tp53_vaf_3,tp53_HGVSp_3,tp53_spot_3,tp53_vt_3,tp53_key_4,tp53_vc_4,tp53_ccf_4,tp53_vaf_4,tp53_HGVSp_4,tp53_spot_4,tp53_vt_4,tp53_key_5,tp53_vc_5,tp53_ccf_5,tp53_vaf_5,tp53_HGVSp_5,tp53_spot_5,tp53_vt_5,tp53_count,tp53_tcn,tp53_mcn,tp53_lcn,tp53_seg_length,tp53_cn_state,tp53_cf,wgd,gene_count,mutation_count,driver_gene_count,driver_mutation_count,snv_driver_mutation_count,indel_driver_mutation_count,max_vaf,tp53_exp_nb_1,tp53_exp_nb_2,tp53_exp_nb_3,tp53_exp_nb_4,tp53_exp_nb_5,tp53_res_1,tp53_res_2,tp53_res_3,tp53_res_4,tp53_res_5,tp53_vc_group_1,tp53_vc_group_2,tp53_vc_group_3,tp53_vc_group_4,tp53_vc_group_5,tp53_vt_group_1,tp53_vt_group_2,tp53_vt_group_3,tp53_vt_group_4,tp53_vt_group_5,tp53_first_group,tp53_group,tp53_res_group,tp53_loh_status,chr_affected,chr_loss,chr_gain,chr_cnloh,frac_genome_altered,tp53_bi_state,tp53_state_wgd,tp53_mut_loh_group,prewgd_tp53_group_1,prewgd_tp53_group_2,T
248,P-0029592-T02-IM6_P-0029592-N01-IM6,P-0029592-T02-IM6,P-0029592,Glioma,Astrocytoma,63.0,Male,Metastasis,0.345908,1.984891,21.140,LIVING,0.00,Stable,4.4,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,61895181,DIPLOID,1.000000,0,5,5,3,3,3,0,0.657581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,4.0,3.0,1.0,0.0,0.164,NaN,NaN,NaN,NaN,NaN,2
879,P-0001882-T03-IM6_P-0001882-N01-IM6,P-0001882-T03-IM6,P-0001882,Glioma,Oligodendroglioma,58.0,Male,Metastasis,0.646441,1.877829,49.447,DECEASED,0.15,Stable,72.9,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,80668474,DIPLOID,1.000000,0,70,83,11,11,10,1,0.477922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,6.0,6.0,0.0,0.0,0.221,NaN,NaN,NaN,NaN,NaN,3
1660,P-0021067-T03-IM6_P-0021067-N01-IM6,P-0021067-T03-IM6,P-0021067,Glioma,Glioblastoma Multiforme,19.0,Male,Metastasis,0.666715,2.006203,19.036,DECEASED,0.11,Stable,4.4,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,80668521,DIPLOID,1.000000,0,5,5,2,2,2,0,0.730924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,5.0,2.0,3.0,0.0,0.201,NaN,NaN,NaN,NaN,NaN,3
1771,P-0013074-T02-IM6_P-0013074-N01-IM6,P-0013074-T02-IM6,P-0013074,Glioma,Glioblastoma Multiforme,55.0,Male,Metastasis,0.857360,2.058942,15.123,DECEASED,0.21,Stable,3.5,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,1,1.0,0.0,29465331,HETLOSS,0.344071,0,4,4,3,3,3,0,0.706941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,LOSS,0_HETLOSS,tp53_res,True,3.0,2.0,1.0,0.0,0.185,NaN,NaN,NaN,NaN,NaN,2
2280,P-0019341-T02-IM6_P-0019341-N01-IM6,P-0019341-T02-IM6,P-0019341,Glioma,Glioblastoma Multiforme,53.0,Female,Metastasis,0.339222,1.832201,23.934,LIVING,0.06,Stable,5.3,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,2.0,0.0,80668632,CNLOH,0.299929,0,6,6,3,3,2,1,0.291297,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,cnLOH,None,None,

In [188]:
master_paired[master_paired['wgd'] == 1]

,Sample_Id,Tumor_Id,Patient_Id,Cancer_Type,Cancer_Type_Detailed,Patient_Current_Age,Sex,Sample_Type,purity,ploidy,Overall_Survival_Months,Overall_Survival_Status,MSI_Score,MSI_Type,TMB_Score,tp53_key_1,tp53_vc_1,tp53_ccf_1,tp53_vaf_1,tp53_HGVSp_1,tp53_spot_1,tp53_vt_1,tp53_key_2,tp53_vc_2,tp53_ccf_2,tp53_vaf_2,tp53_HGVSp_2,tp53_spot_2,tp53_vt_2,tp53_key_3,tp53_vc_3,tp53_ccf_3,tp53_vaf_3,tp53_HGVSp_3,tp53_spot_3,tp53_vt_3,tp53_key_4,tp53_vc_4,tp53_ccf_4,tp53_vaf_4,tp53_HGVSp_4,tp53_spot_4,tp53_vt_4,tp53_key_5,tp53_vc_5,tp53_ccf_5,tp53_vaf_5,tp53_HGVSp_5,tp53_spot_5,tp53_vt_5,tp53_count,tp53_tcn,tp53_mcn,tp53_lcn,tp53_seg_length,tp53_cn_state,tp53_cf,wgd,gene_count,mutation_count,driver_gene_count,driver_mutation_count,snv_driver_mutation_count,indel_driver_mutation_count,max_vaf,tp53_exp_nb_1,tp53_exp_nb_2,tp53_exp_nb_3,tp53_exp_nb_4,tp53_exp_nb_5,tp53_res_1,tp53_res_2,tp53_res_3,tp53_res_4,tp53_res_5,tp53_vc_group_1,tp53_vc_group_2,tp53_vc_group_3,tp53_vc_group_4,tp53_vc_group_5,tp53_vt_group_1,tp53_vt_group_2,tp53_vt_group_3,tp53_vt_group_4,tp53_vt_group_5,tp53_first_group,tp53_group,tp53_res_group,tp53_loh_status,chr_affected,chr_loss,chr_gain,chr_cnloh,frac_genome_altered,tp53_bi_state,tp53_state_wgd,tp53_mut_loh_group,prewgd_tp53_group_1,prewgd_tp53_group_2,T
2,P-0023546-T01-IM6_P-0023546-N01-IM6,P-0023546-T01-IM6,P-0023546,Prostate Cancer,Prostate Neuroendocrine Carcinoma,50.0,Male,Primary,0.865628,3.115253,4.800,DECEASED,2.37,Stable,3.5,P-0023546-T01-IM6-17_7578442_T_C,Missense_Mutation,0.933,0.845070,p.Tyr163Cys,163,SNP,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,1,3,3.0,0.0,25250470,CNLOH BEFORE & LOSS,0.835504,1,3,3,1,1,1,0,0.845070,2.797574,NaN,NaN,NaN,NaN,0.202426,NaN,NaN,NaN,NaN,missense,None,None,None,None,SNV,None,None,None,None,NaN,NaN,NaN,True,18.0,13.0,4.0,4.0,0.836,bi,bi,1_loh,bi,bi,1
3,P-0023546-T02-IM6_P-0023546-N01-IM6,P-0023546-T02-IM6,P-0023546,Prostate Cancer,Prostate Adenocarcinoma,50.0,Male,Primary,0.312907,3.136841,4.800,DECEASED,0.82,Stable,2.6,P-0023546-T02-IM6-17_7578442_T_C,Missense_Mutation,1.000,0.636735,p.Tyr163Cys,163,SNP,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,1,2,2.0,0.0,25237770,LOSS BEFORE,0.312907,1,2,2,1,1,1,0,0.636735,4.069805,NaN,NaN,NaN,NaN,-2.069805,NaN,NaN,NaN,NaN,missense,None,None,None,None,SNV,None,None,None,None,NaN,NaN,NaN,True,18.0,13.0,5.0,1.0,0.775,bi,bi,1_loh,bi,bi,2
5,P-0032161-T01-IM6_P-0032161-N01-IM6,P-0032161-T01-IM6,P-0032161,Endometrial Cancer,Endometrial Carcinoma,69.0,Female,Metastasis,0.351382,3.211227,1.874,LIVING,0.00,Stable,7.0,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,2.0,0.0,11930051,LOSS BEFORE,0.301464,1,8,8,3,3,2,1,0.411150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,True,18.0,16.0,4.0,1.0,0.674,no_bi,mono,0_loh,tp53_res,mono,1
7,P-0002578-T01-IM3_P-0002578-N01-IM3,P-0002578-T01-IM3,P-0002578,Peripheral Nervous System,Neuroblastoma,18.0,Female,Primary,0.872614,2.870812,61.216,DECEASED,1.43,Stable,3.3,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,80668564,DOUBLE LOSS AFTER,1.000000,1,3,3,3,3,2,1,0.413943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,False,18.0,16.0,1.0,2.0,0.820,no_bi,2WT,0_noloh,tp53_res,2WT,1
32,P-0032859-T01-IM6_P-0032859-N01-IM6,P-0032859-T01-IM6,P-0032859,Small Cell Lung Cancer,Small Cell Lung Cancer,73.0,Male,Metastasis,0.571983,3.583489,18.575,LIVING,0.91,Stable,7.9,P-0032859-T01-IM6-17_7577551_C_G,Missense_Mutation,0.929,0.531507,p.Gly244Arg,244,SNP,None,None,NaN,NaN,None,None,None,None,Non

In [84]:
h=[2,1,3]
h.sort()
h

[1, 2, 3]

In [98]:
master_paired[master_paired['Patient_Id'] == 'P-0000037']

,Sample_Id,Tumor_Id,Patient_Id,Cancer_Type,Cancer_Type_Detailed,Patient_Current_Age,Sex,Sample_Type,purity,ploidy,Overall_Survival_Months,Overall_Survival_Status,MSI_Score,MSI_Type,TMB_Score,tp53_key_1,tp53_vc_1,tp53_ccf_1,tp53_vaf_1,tp53_HGVSp_1,tp53_spot_1,tp53_vt_1,tp53_key_2,tp53_vc_2,tp53_ccf_2,tp53_vaf_2,tp53_HGVSp_2,tp53_spot_2,tp53_vt_2,tp53_key_3,tp53_vc_3,tp53_ccf_3,tp53_vaf_3,tp53_HGVSp_3,tp53_spot_3,tp53_vt_3,tp53_key_4,tp53_vc_4,tp53_ccf_4,tp53_vaf_4,tp53_HGVSp_4,tp53_spot_4,tp53_vt_4,tp53_key_5,tp53_vc_5,tp53_ccf_5,tp53_vaf_5,tp53_HGVSp_5,tp53_spot_5,tp53_vt_5,tp53_count,tp53_tcn,tp53_mcn,tp53_lcn,tp53_seg_length,tp53_cn_state,tp53_cf,wgd,gene_count,mutation_count,driver_gene_count,driver_mutation_count,snv_driver_mutation_count,indel_driver_mutation_count,max_vaf,tp53_exp_nb_1,tp53_exp_nb_2,tp53_exp_nb_3,tp53_exp_nb_4,tp53_exp_nb_5,tp53_res_1,tp53_res_2,tp53_res_3,tp53_res_4,tp53_res_5,tp53_vc_group_1,tp53_vc_group_2,tp53_vc_group_3,tp53_vc_group_4,tp53_vc_group_5,tp53_vt_group_1,tp53_vt_group_2,tp53_vt_group_3,tp53_vt_group_4,tp53_vt_group_5,tp53_first_group,tp53_group,tp53_res_group,tp53_loh_status,chr_affected,chr_loss,chr_gain,chr_cnloh,frac_genome_altered,tp53_bi_state,tp53_state_wgd,tp53_mut_loh_group,prewgd_tp53_group_1,prewgd_tp53_group_2,T
11312,P-0000037-T01-IM3_P-0000037-N01-IM3,P-0000037-T01-IM3,P-0000037,Hepatobiliary Cancer,Hepatocellular Carcinoma,70.0,Male,Metastasis,NaN,1.911712,71.079,LIVING,0.08,Stable,4.5,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,1,1.0,0.0,15947750,HETLOSS,0.402369,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,LOSS,0_HETLOSS,tp53_res,True,6.0,6.0,0.0,0.0,0.290,NaN,NaN,NaN,NaN,NaN,1
11313,P-0000037-T02-IM3_P-0000037-N01-IM3,P-0000037-T02-IM3,P-0000037,Hepatobiliary Cancer,Hepatocellular Carcinoma,70.0,Male,Primary,NaN,2.170814,71.079,LIVING,0.27,Stable,3.3,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,None,None,NaN,NaN,None,None,None,0,2,1.0,1.0,80668359,DIPLOID,1.000000,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,WILD_TYPE,None,None,False,9.0,6.0,3.0,2.0,0.412,NaN,NaN,NaN,NaN,NaN,2
